In [10]:
import pandas as pd
df = pd.read_csv('data/u.data', sep='\t', 
                names=['user_id', 'item_id', 'rating', 'timestamp'])

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
